In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords

In [5]:
train = pd.read_csv("IMDB Dataset.csv", header=0, quoting=2)

In [6]:
#数据清洗和预处理
def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review).get_text()    
    lettera_only1 = re.sub("\.",'. ',review_text)
    letters_only = re.sub("[^a-zA-Z.]", " ", lettera_only1) 
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if w not in stops]   
    return( " ".join( meaningful_words ))
num_reviews = train["review"].size
clean_reviews = []
for i in range( 0, num_reviews ):
    clean_reviews.append( review_to_words( train["review"][i] ) )
clean_train_reviews_wordlist = []
num_train_reviews = 30000
for i in range(0,num_train_reviews):
    sentences = clean_reviews[i].split('. ')
    #print(sentences)
    sentences = [s.split() for s in sentences]
    clean_train_reviews_wordlist += sentences
    #print(i)
    

In [7]:
from gensim.models import Word2Vec

In [8]:
w2v_model = Word2Vec(clean_train_reviews_wordlist,vector_size = 32, window=5, min_count=10, workers=4)
print(w2v_model)

Word2Vec(vocab=21969, vector_size=32, alpha=0.025)


In [9]:
sims = w2v_model.wv.most_similar('happy', topn=15)

In [10]:
w2v_model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [11]:
def Make_data(data,clean_views):
    Y1 = data['sentiment']
    s = len(Y1)
    Y1[Y1 == 'positive'] = 1
    Y1[Y1 == 'negative'] = 0
    Y1 = Y1.values
    Y = np.zeros([1,s])
    Y = Y + Y1
    Y = Y.astype(float)
    X = np.zeros([s,32])
    for i in range(s):
        paragraph = re.sub("\.",' ',clean_views[i])
        wordlist = paragraph.split()
        cnt = 0
        for word in wordlist:
            try:
                X[i] += w2v_model.wv[word]
                cnt += 1
            except:
                continue
        X[i] /= cnt
    X = X.T
    return X,Y

In [12]:
X,y = Make_data(train,clean_reviews)

In [13]:
X.shape,y.shape

((32, 50000), (1, 50000))

In [14]:
X = X.T
y = y.T

In [15]:
np.savetxt("X.txt", X, fmt="%d", delimiter=",")
np.savetxt("y.txt", y, fmt="%d", delimiter=",")

In [17]:
X.shape

(50000, 32)